**Table of contents**<a id='toc0_'></a>    
- [Imports](#toc1_)    
- [Load Data](#toc2_)    
- [Check missing values](#toc3_)    
- [Preprocessing](#toc4_)    
- [Training](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
# !pip install pytorch-nlp
# !pip install torchviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4149 sha256=229fab9f9b4be6eac87cab9f3eaa4961d0047e496b1f44fb8d9da25eeb92257b
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


# <a id='toc1_'></a>[Imports](#toc0_)

In [3]:
%reset -f
%load_ext autoreload
%autoreload 2

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nltk.corpus import stopwords
from nltk import word_tokenize

from tqdm.notebook import tqdm

import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchnlp.encoders.text import TreebankEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import f1_score
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

USE_SAVED_INPUTS = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/kaggle/input/asrs-aeroguard/01_df_train_val_test.pkl


device(type='cuda')

# <a id='toc2_'></a>[Load Data](#toc0_)

In [5]:
filepath = fr'/kaggle/input/asrs-aeroguard/01_df_train_val_test.pkl'
df_train_val_test = pd.read_pickle(filepath)
df_train_val_test

,ACN_NUM_ACN,TIME_DATE,TIME_1_LOCAL_TIME_OF_DAY,PLACE_LOCALE_REFERENCE,PLACE_1_STATE_REFERENCE,PLACE_2_RELATIVE_POSITION_ANGLE_RADIAL,PLACE_3_RELATIVE_POSITION_DISTANCE_NAUTICAL_MILES,PLACE_4_ALTITUDE_AGL_SINGLE_VALUE,PLACE_5_ALTITUDE_MSL_SINGLE_VALUE,ENVIRONMENT_FLIGHT_CONDITIONS,...,ASSESSMENTS_CONTRIBUTING_FACTORS_SITUATIONS,ASSESSMENTS_1_PRIMARY_PROBLEM,REPORT_1_NARRATIVE,REPORT_1_1_CALLBACK,REPORT_2_NARRATIVE,REPORT_2_1_CALLBACK,REPORT_1_2_SYNOPSIS,TRAIN_VAL_TEST_SPLIT,EVENT_RISK,EVENT_RISK_STR
0,1574675,201808,0601-1200,SNA.Airport,CA,NaN,NaN,NaN,5000.0,NA,...,Human Factors,Human Factors,SNA RNP-Z to Runway 20R. The FMC was properly ...,NA,We were cleared for the RNP RNAV Z 20R Approac...,NA,B737-700 flight crew reported failing to make ...,Train,2,Medium risk
1,1224894,201412,0601-1200,MSY.Airport,LA,NaN,NaN,1000.0,NaN,VMC,...,Human Factors,Human Factors,On base to final turn to runway 1 in MSY at ap...,NA,NA,NA,Captain reports sighting of a drone at 1;000 f...,Train,0,Low risk
2,1134202,201312,1201-1800,ZZZ.ARTCC,US,NaN,NaN,NaN,2600.0,IMC,...,Human Factors; Aircraft; Procedure; Weather,Aircraft,I climbed to my filed altitude of 5;000 FT; an...,NA,NA,NA,SR22 pilot became disoriented on approach in I...,Train,3,Moderately high risk
3,1222074,201411,1201-1800,CWA.Airport,WI,NaN,20.0,NaN,4000.0,NA,...,Aircraft,Aircraft,I had my pitot heat checked prior to winter an...,NA,NA,NA,BE58 pilot experiences pitot heat failure desc...,Train,2,Medium risk
4,1733019,202003,1801-2400,ZDV.ARTCC,CO,NaN,NaN,NaN,32000.0,NA,...,Airspace Structure; Weather,Weather,At 32000 ft. just north of PUB the aircraft ex...,NA,NA,NA,B737 First Officer reported unexpected moderat...,Train,3,Moderately high risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47718,1341108,201603,0601-1200,ZZZ.Airport,US,NaN,NaN,NaN,NaN,IMC,...,Company Policy; Human Factors,Human Factors,We did an originator out of ZZZ and had a main...,NA,[Report narrative contained no additional info...,NA,CRJ-900 flight crew reported being dispatched ...,Test,3,Moderately high risk
47719,1087474,201305,1201-1800,PHX.Airport,AZ,NaN,NaN,NaN,8000.0,VMC,...,Aircraft; Human Factors; Procedure,Ambiguous,PHX takeoff Runway 25R flaps 5. CLEARANCE: MAX...,NA,There was an Airbus that departed before us an...,NA,CE750 flight crew departing PHX Runway 25R on ...,Test,3,Moderately high risk
47720,1756601,202008,NA,NA,NA,NaN,NaN,0.0,NaN,NA,...,Environment - Non Weather Related; Company Pol...,Company Policy,I was scheduled to complete the one day traini...,NA,NA,NA,Air carrier First Officer reported that re-qua...,Test,0,Low risk
47721,1102938,201307,0601-1200,MEM.Airport,TN,NaN,NaN,NaN,NaN,VMC,...,Airspace Structure; Chart Or Publication; Proc...,Human Factors,We departed Runway 36C in MEM via the GOETZ TW...,NA,NA,NA,On initial climb out via the GOETZ RNAV SID fr...,Test,2,Medium risk


# <a id='toc3_'></a>[Check missing values](#toc0_)

In [6]:
# There are 3 entries with "[Report narrative contained no additional information.]" which technically means
# a missing entry. However, because the number is so low, we will not preprocess it.
df_train_val_test['REPORT_1_NARRATIVE'].value_counts().head()

[Report narrative contained no additional information.]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [7]:
tmp_ser = df_train_val_test['REPORT_1_NARRATIVE'].str.lower()
tmp_ser

0        sna rnp-z to runway 20r. the fmc was properly ...
1        on base to final turn to runway 1 in msy at ap...
2        i climbed to my filed altitude of 5;000 ft; an...
3        i had my pitot heat checked prior to winter an...
4        at 32000 ft. just north of pub the aircraft ex...
                               ...                        
47718    we did an originator out of zzz and had a main...
47719    phx takeoff runway 25r flaps 5. clearance: max...
47720    i was scheduled to complete the one day traini...
47721    we departed runway 36c in mem via the goetz tw...
47722    walking to the aircraft; the inbound crew info...
Name: REPORT_1_NARRATIVE, Length: 47723, dtype: object

In [8]:
tmp_ser = tmp_ser.str.replace(r'[\W\d_]+', ' ', regex=True)
tmp_ser

0        sna rnp z to runway r the fmc was properly pro...
1        on base to final turn to runway in msy at appr...
2        i climbed to my filed altitude of ft and was r...
3        i had my pitot heat checked prior to winter an...
4        at ft just north of pub the aircraft experienc...
                               ...                        
47718    we did an originator out of zzz and had a main...
47719    phx takeoff runway r flaps clearance maxxo cnx...
47720    i was scheduled to complete the one day traini...
47721    we departed runway c in mem via the goetz two ...
47722    walking to the aircraft the inbound crew infor...
Name: REPORT_1_NARRATIVE, Length: 47723, dtype: object

In [9]:
stopword_list = stopwords.words('english')
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stopword_list]
    return ' '.join(filtered_tokens)

# Apply stopwords removal to the Series
tmp_ser = tmp_ser.apply(remove_stopwords)
tmp_ser

0        sna rnp z runway r fmc properly programmed mcp...
1        base final turn runway msy approximately feet ...
2        climbed filed altitude ft right top clouds kin...
3        pitot heat checked prior winter prior flights ...
4        ft north pub aircraft experienced moderate sev...
                               ...                        
47718    originator zzz maintenance delay going back fo...
47719    phx takeoff runway r flaps clearance maxxo cnx...
47720    scheduled complete one day training event x ty...
47721    departed runway c mem via goetz two departure ...
47722    walking aircraft inbound crew informed cabin w...
Name: REPORT_1_NARRATIVE, Length: 47723, dtype: object

In [10]:
df_train_val_test['REPORT_1_NARRATIVE'] = tmp_ser
df_train_val_test['REPORT_1_NARRATIVE']

0        sna rnp z runway r fmc properly programmed mcp...
1        base final turn runway msy approximately feet ...
2        climbed filed altitude ft right top clouds kin...
3        pitot heat checked prior winter prior flights ...
4        ft north pub aircraft experienced moderate sev...
                               ...                        
47718    originator zzz maintenance delay going back fo...
47719    phx takeoff runway r flaps clearance maxxo cnx...
47720    scheduled complete one day training event x ty...
47721    departed runway c mem via goetz two departure ...
47722    walking aircraft inbound crew informed cabin w...
Name: REPORT_1_NARRATIVE, Length: 47723, dtype: object

# <a id='toc4_'></a>[Preprocessing](#toc0_)

In [11]:
def preprocess_inputs(df):
    df = df.copy()
    
    df_train_val_test['REPORT_1_NARRATIVE'].str.lower()
    
    df_train = df.query('TRAIN_VAL_TEST_SPLIT == "Train"')
    df_val = df.query('TRAIN_VAL_TEST_SPLIT == "Validation"')
    df_test = df.query('TRAIN_VAL_TEST_SPLIT == "Test"')
    
    X_train = df_train['REPORT_1_NARRATIVE']
    X_val = df_val['REPORT_1_NARRATIVE']
    X_test = df_test['REPORT_1_NARRATIVE']
    
    y_train = torch.tensor(df_train['EVENT_RISK'].values, dtype=int)
    y_val = torch.tensor(df_val['EVENT_RISK'].values, dtype=int)
    y_test = torch.tensor(df_test['EVENT_RISK'].values, dtype=int)
    
    print('Fit encoder ...')
    encoder = TreebankEncoder(X_train)
    vocab_len = len(encoder.vocab)
    
    print('Start encoding ...')
    X_train = encoder.batch_encode(X_train).tensor
    X_val = encoder.batch_encode(X_val).tensor
    X_test = encoder.batch_encode(X_test).tensor
    
    print('Fix padding ...')
    max_seq_len = X_train.shape[1]
    # Pad shorter sequences to length of max_seq_len
    # and cut longer sequences to length of max_seq_len
    X_val = pad_sequence(X_val, batch_first=True, padding_value=0)
    X_val = torch.nn.functional.pad(X_val, (0, max_seq_len - X_val.shape[1]))
    
    X_test = pad_sequence(X_test, batch_first=True, padding_value=0)
    X_test = torch.nn.functional.pad(X_test, (0, max_seq_len - X_test.shape[1]))
    
    return X_train, y_train, X_val, y_val, X_test, y_test, vocab_len, max_seq_len

In [12]:
X_train, y_train, X_val, y_val, X_test, y_test, VOCAB_LEN, MAX_SEQ_LEN = preprocess_inputs(df_train_val_test)

print(f'{X_train.shape=}, {y_train.shape=}')
print(f'{X_val.shape=}, {y_val.shape=}')
print(f'{X_test.shape=}, {y_test.shape=}')

print(f'{VOCAB_LEN=}')
print(f'{MAX_SEQ_LEN=}')

Fit encoder ...
[nltk_data] Downloading package perluniprops to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
Start encoding ...
Fix padding ...
X_train.shape=torch.Size([38655, 1165]), y_train.shape=torch.Size([38655])
X_val.shape=torch.Size([4295, 1165]), y_val.shape=torch.Size([4295])
X_test.shape=torch.Size([4773, 1165]), y_test.shape=torch.Size([4773])
VOCAB_LEN=39349
MAX_SEQ_LEN=1165


In [13]:
if not USE_SAVED_INPUTS:
    torch.save(X_train, '/kaggle/working/X_train.pt')
    torch.save(y_train, '/kaggle/working/y_train.pt')
    torch.save(X_val, '/kaggle/working/X_val.pt')
    torch.save(y_val, '/kaggle/working/y_val.pt')
    torch.save(X_test, '/kaggle/working/X_test.pt')
    torch.save(y_test, '/kaggle/working/y_test.pt')
    np.save('/kaggle/working/VOCAB_LEN.npy', VOCAB_LEN)
    np.save('/kaggle/working/MAX_SEQ_LEN.npy', MAX_SEQ_LEN)
else:
    X_train = torch.load('/kaggle/working/X_train.pt')
    y_train = torch.load('/kaggle/working/y_train.pt')
    X_val = torch.load('/kaggle/working/X_val.pt')
    y_val = torch.load('/kaggle/working/y_val.pt')
    X_test = torch.load('/kaggle/working/X_test.pt')
    y_test = torch.load('/kaggle/working/y_test.pt')
    VOCAB_LEN = np.load('/kaggle/working/VOCAB_LEN.npy')
    MAX_SEQ_LEN = np.load('/kaggle/working/MAX_SEQ_LEN.npy')

# <a id='toc5_'></a>[Training](#toc0_)

In [50]:
class MyModel(nn.Module):
    def __init__(self, vocab_length, max_seq_length, num_classes, embedding_dim=128, hidden_dim=256):
        super(MyModel, self).__init__()

        self.word_embedding = nn.Embedding(vocab_length, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=4, batch_first=True, bidirectional=True)
        self.dense = nn.Linear(2 * hidden_dim, num_classes)

    def forward(self, inputs):
        word_embedding = self.word_embedding(inputs)
        lstm_output, (lstm_hidden, _) = self.lstm(word_embedding)
        lstm_output_concat = torch.cat((lstm_hidden[-2, :, :], lstm_hidden[-1, :, :]), dim=1)  # Concatenate the last hidden states of both directions
        output = self.dense(lstm_output_concat)
        return output

In [53]:
learning_rate = 0.001
batch_size = 256
num_epochs = 100
n_classes = 5

model = MyModel(VOCAB_LEN, MAX_SEQ_LEN, n_classes, embedding_dim=256, hidden_dim=8).to(device)

def focal_loss(inputs, targets, alpha, gamma):
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    bce_loss = criterion(inputs, targets)
    pt = torch.exp(-bce_loss)
    focal_loss = alpha * (1 - pt) ** gamma * bce_loss
    return focal_loss.mean()

alpha = 1.25
gamma = 2.0

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(val_dataset, batch_size=batch_size)

for epoch in range(num_epochs):
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Learning rate: {current_lr:.6f}')
    
    model.train()
    train_loss = 0.0
    train_labels = []
    train_predictions = []

    pbar = tqdm(train_loader, total=len(train_loader))
    for inputs, labels in pbar:
        inputs = inputs.to(device)
        labels = labels.to(device)

        labels_one_hot = F.one_hot(labels, n_classes).float()

        outputs = model(inputs)
        loss = focal_loss(outputs, labels_one_hot, alpha, gamma)  # Focal Loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_labels.extend(labels.tolist())
        _, predicted = torch.max(outputs.data, 1)
        train_predictions.extend(predicted.tolist())

        train_loss += loss.item() * inputs.size(0)
        pbar.set_description(f'Training loss: {loss.item()=}')

    model.eval()
    eval_loss = 0.0
    eval_labels = []
    eval_predictions = []

    with torch.no_grad():
        for inputs, labels in eval_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            labels_one_hot = F.one_hot(labels, n_classes).float()

            outputs = model(inputs)
            loss = focal_loss(outputs, labels_one_hot, alpha, gamma)

            eval_labels.extend(labels.tolist())
            _, predicted = torch.max(outputs.data, 1)
            eval_predictions.extend(predicted.tolist())

            eval_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_dataset)
    train_f1 = f1_score(train_labels, train_predictions, average='macro')

    eval_loss /= len(val_dataset)
    eval_f1 = f1_score(eval_labels, eval_predictions, average='macro')

    scheduler.step(eval_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}]: '
          f'Train Loss: {train_loss:.4f}, Train F1: {train_f1:.4f}, '
          f'Eval Loss: {eval_loss:.4f}, Eval F1: {eval_f1:.4f}')

Learning rate: 0.001000


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch [1/100]: Train Loss: 0.1703, Train F1: 0.1348, Eval Loss: 0.1556, Eval F1: 0.1032
Learning rate: 0.001000


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch [2/100]: Train Loss: 0.1520, Train F1: 0.1533, Eval Loss: 0.1471, Eval F1: 0.2000
Learning rate: 0.001000


...

Epoch [16/100]: Train Loss: 0.1061, Train F1: 0.6185, Eval Loss: 0.1546, Eval F1: 0.3997
Learning rate: 0.000010


  0%|          | 0/151 [00:00<?, ?it/s]

Epoch [17/100]: Train Loss: 0.1060, Train F1: 0.6193, Eval Loss: 0.1548, Eval F1: 0.4027
Learning rate: 0.000010


  0%|          | 0/151 [00:00<?, ?it/s]

KeyboardInterrupt: 